In [18]:
import stim
import pymatching as pm
import numpy as np

def build_repetition_circuit(d=3, rounds=3, p1=0.0, p2=0.0, pm=0.02):
    """
    Minimal repetition-code style demo:
    - d data qubits, d-1 ancillas that measure Z-parity (via CZ).
    - Injects 1q and 2q depolarizing noise and measurement flip noise.
    - DETECTORs are time-differences of the same parity measurement across rounds.
    - OBSERVABLE_INCLUDE is a simple placeholder tied to one ancilla meas (toy).
      (Good enough to test Stim->DEM->PyMatching; not a rigorous logical.)
    """
    n_data = d
    n_anc = d - 1
    c = stim.Circuit()

    # Reset all qubits to |0>
    c.append("R", range(n_data + n_anc))

    for t in range(rounds):
        # (Optional) 1q idle noise on data & ancillas
        if p1 > 0:
            for q in range(n_data + n_anc):
                c.append("DEPOLARIZE1", [q], p1)

        # Parity measurement using ancilla a with neighbors j and j+1
        for j in range(n_anc):
            a = n_data + j
            dL, dR = j, j+1

            # Noisy entangling with 2q noise
            c.append("CZ", [a, dL])
            if p2 > 0:
                c.append("DEPOLARIZE2", [a, dL], p2)
            c.append("CZ", [a, dR])
            if p2 > 0:
                c.append("DEPOLARIZE2", [a, dR], p2)

        # Measure ancillas with measurement-flip noise
        for j in range(n_anc):
            a = n_data + j
            if pm > 0:
                c.append("X_ERROR", [a], pm)  # model readout flip as pre-meas X
            c.append("M", [a])

        # Create detectors as time-differences of the *same* ancilla between rounds
        for j in range(n_anc):
            # current round's j-th ancilla measurement is rec offset -(j+1)
            cur = stim.target_rec(-(j + 1))
            if t == 0:
                # first round: no previous measurement exists; use raw meas as detector
                c.append("DETECTOR", [cur])
            else:
                # previous round's same ancilla is n_anc further back
                prev = stim.target_rec(-(n_anc + j + 1))
                c.append("DETECTOR", [prev, cur])

    # Simple placeholder observable: tie it to the last ancilla measurement
    # (This makes the pipeline produce observable flips; for *proper* logicals,
    # we’ll switch to a standard construction later.)
    c.append("OBSERVABLE_INCLUDE", [stim.target_rec(-1)], 0)

    return c

# Try higher noise so flips show up
circuit = build_repetition_circuit(d=3, rounds=6, p1=0.002, p2=0.01, pm=0.02)
print(circuit)

shots = 50_000

# Build DEM -> matcher
dem = circuit.detector_error_model(decompose_errors=True)
matcher = pm.Matching.from_detector_error_model(dem)

# Sample detectors/observables
sampler = circuit.compile_detector_sampler()
dets, obs = sampler.sample(shots, separate_observables=True)

# Decode and estimate "logical" (toy) error rate
pred = matcher.decode_batch(dets).reshape(-1, 1)
errors = np.count_nonzero(pred ^ obs)
p_L = errors / shots
p_L


R 0 1 2 3 4
DEPOLARIZE1(0.002) 0 1 2 3 4
CZ 3 0
DEPOLARIZE2(0.01) 3 0
CZ 3 1
DEPOLARIZE2(0.01) 3 1
CZ 4 1
DEPOLARIZE2(0.01) 4 1
CZ 4 2
DEPOLARIZE2(0.01) 4 2
X_ERROR(0.02) 3
M 3
X_ERROR(0.02) 4
M 4
DETECTOR rec[-1]
DETECTOR rec[-2]
DEPOLARIZE1(0.002) 0 1 2 3 4
CZ 3 0
DEPOLARIZE2(0.01) 3 0
CZ 3 1
DEPOLARIZE2(0.01) 3 1
CZ 4 1
DEPOLARIZE2(0.01) 4 1
CZ 4 2
DEPOLARIZE2(0.01) 4 2
X_ERROR(0.02) 3
M 3
X_ERROR(0.02) 4
M 4
DETECTOR rec[-3] rec[-1]
DETECTOR rec[-4] rec[-2]
DEPOLARIZE1(0.002) 0 1 2 3 4
CZ 3 0
DEPOLARIZE2(0.01) 3 0
CZ 3 1
DEPOLARIZE2(0.01) 3 1
CZ 4 1
DEPOLARIZE2(0.01) 4 1
CZ 4 2
DEPOLARIZE2(0.01) 4 2
X_ERROR(0.02) 3
M 3
X_ERROR(0.02) 4
M 4
DETECTOR rec[-3] rec[-1]
DETECTOR rec[-4] rec[-2]
DEPOLARIZE1(0.002) 0 1 2 3 4
CZ 3 0
DEPOLARIZE2(0.01) 3 0
CZ 3 1
DEPOLARIZE2(0.01) 3 1
CZ 4 1
DEPOLARIZE2(0.01) 4 1
CZ 4 2
DEPOLARIZE2(0.01) 4 2
X_ERROR(0.02) 3
M 3
X_ERROR(0.02) 4
M 4
DETECTOR rec[-3] rec[-1]
DETECTOR rec[-4] rec[-2]
DEPOLARIZE1(0.002) 0 1 2 3 4
CZ 3 0
DEPOLARIZE2(0.01) 3 0
CZ 3 1

np.float64(0.0)